In [198]:
import pandas as pd
import os
from helpers.memory import memory_mib

In [199]:
data_path = os.path.join('..', 'data', 'raw')

In [200]:
master = pd.read_csv(os.path.join(data_path, 'Master.csv'))

In [201]:
master.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7761 entries, 0 to 7760
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   playerID      7520 non-null   object 
 1   coachID       395 non-null    object 
 2   hofID         366 non-null    object 
 3   firstName     7748 non-null   object 
 4   lastName      7761 non-null   object 
 5   nameNote      18 non-null     object 
 6   nameGiven     5985 non-null   object 
 7   nameNick      1306 non-null   object 
 8   height        7334 non-null   float64
 9   weight        7336 non-null   float64
 10  shootCatch    7048 non-null   object 
 11  legendsID     6577 non-null   object 
 12  ihdbID        7125 non-null   float64
 13  hrefID        7457 non-null   object 
 14  firstNHL      6851 non-null   float64
 15  lastNHL       6851 non-null   float64
 16  firstWHA      903 non-null    float64
 17  lastWHA       903 non-null    float64
 18  pos           7447 non-null 

In [202]:
(master['playerID']
 .pipe(pd.isnull)  # pipy czyli lejki
 .value_counts(normalize=True))

playerID
False    0.968947
True     0.031053
Name: proportion, dtype: float64

In [203]:
master_original = master.copy() # dobrze sobie zrobić kopię

In [204]:
 # usuwamy wiersze z brakującymi danymi
master.dropna(subset=['playerID'], inplace=True)

In [205]:
master.shape

(7520, 31)

In [206]:
# pipe to jest metoda, która pozwala na łańcuchowe wywoływanie funkcji
# można sobie fajnie, policzyć ile jest brakujących danych
(master['playerID']
 .pipe(pd.isnull)  # pipy czyli lejki
 .value_counts())


playerID
False    7520
Name: count, dtype: int64

In [207]:
# usuwamy wiersze z brakującymi danymi
master.dropna(subset=['firstNHL', 'lastNHL'], how='all', inplace=True)

In [208]:
master.shape

(6851, 31)

In [209]:
# query alternatywa do filtrowania z .loc
master.query("lastNHL >= 1980").head()

,playerID,coachID,hofID,firstName,lastName,nameNote,nameGiven,nameNick,height,weight,...,birthDay,birthCountry,birthState,birthCity,deathYear,deathMon,deathDay,deathCountry,deathState,deathCity
0,aaltoan01,NaN,NaN,Antti,Aalto,NaN,Antti,NaN,73.0,210.0,...,4.0,Finland,NaN,Lappeenranta,NaN,NaN,NaN,NaN,NaN,NaN
4,abdelju01,NaN,NaN,Justin,Abdelkader,NaN,NaN,NaN,73.0,195.0,...,25.0,USA,MI,Muskegon,NaN,NaN,NaN,NaN,NaN,NaN
9,abidra01,NaN,NaN,Ramzi,Abid,NaN,NaN,NaN,74.0,210.0,...,24.0,Canada,QC,Montreal,NaN,NaN,NaN,NaN,NaN,NaN
11,abrahth01,NaN,NaN,Thommy,Abrahamsson,NaN,Thommy Ulf,NaN,74.0,185.0,...,12.0,Sweden,NaN,Leksand,NaN,NaN,NaN,NaN,NaN,NaN
14,actonke01,NaN,NaN,Keith,Acton,NaN,Keith Edward,Woody,68.0,170.0,...,15.0,Canada,ON,Stouffville,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
# najczęściej używane filtrowanie
master = master.loc[master['lastNHL'] >= 1980]

In [1]:
# współczynnik correlacji Pearsona, Kendala, Spearmana
# mierzy liniową zależność między dwiema zmiennymi
# -1 korelacja ujemna, 0 brak korelacji, 1 korelacja dodatnia
# master.corr()

In [212]:
columns_to_keep = ["playerID", "firstName", "lastName",
                   "pos", "birthYear", "birthMon", "birthDay",
                   "birthCountry", "birthState", "birthCity"]

In [213]:
# odsiewamy kolumny, które nas interesują
master[columns_to_keep].shape

(4627, 10)

[101Regex](https://regex101.com/)

In [214]:
# odsiewamy kolumny, z pomocą filter z regex, przydatne do zbiorów z dużą ilością kolumn(może być ich 500+)
master.filter(regex='(playerID|pos|^birth)|(Name$)').shape

(4627, 10)

In [215]:
# filtrowanie zwraca nowy df z wybranymi kolumnami, więc trzeba przypisać do zmiennej
master = master[columns_to_keep]

In [216]:
master.memory_usage()

Index           37016
playerID        37016
firstName       37016
lastName        37016
pos             37016
birthYear       37016
birthMon        37016
birthDay        37016
birthCountry    37016
birthState      37016
birthCity       37016
dtype: int64

In [217]:
memory_mib(master)

'0.39 MiB'

In [218]:
memory_mib(master_original)

'1.84 MiB'

## Dane kategoryczne = przyjmują skończoną liczbę wartości, które nie mają sensu numerycznego

In [219]:
# przykłąd danych kategorycznych
master['pos'].value_counts()

pos
D      1418
C      1037
L       848
R       832
G       463
F        28
L/C       1
Name: count, dtype: int64

In [220]:
# stworzenie kolumny kategorycznej, zmniejsza zużycie pamięci
pd.Categorical(master['pos'])

['C', 'L', 'L', 'D', 'C', ..., 'R', 'L', 'L', 'C', 'D']
Length: 4627
Categories (7, object): ['C', 'D', 'F', 'G', 'L', 'L/C', 'R']

Warning
[Copy-on-write](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)

In [221]:
# dane numeryczne nie ciągłe
# dane kategoryczne nominalne
# cztery typy danych
# nominalne, porządkowe, interwałowe, ilorazowe

master.loc[:, 'pos'] = pd.Categorical(master['pos'])

In [222]:
master.memory_usage()

Index           37016
playerID        37016
firstName       37016
lastName        37016
pos             37016
birthYear       37016
birthMon        37016
birthDay        37016
birthCountry    37016
birthState      37016
birthCity       37016
dtype: int64

In [224]:
# dane kategoryczne oszczędzają pamięć
master.loc[:, 'birthCountry'] = pd.Categorical(master['birthCountry'])
master.loc[:, 'birthState'] = pd.Categorical(master['birthState'])

In [223]:
memory_mib(master)

'0.39 MiB'

In [226]:
master.set_index('playerID', inplace=True)

In [227]:
master.dtypes

firstName        object
lastName         object
pos              object
birthYear       float64
birthMon        float64
birthDay        float64
birthCountry     object
birthState       object
birthCity        object
dtype: object

In [225]:
pd.Categorical(master['birthCountry'])

['Finland', 'USA', 'Canada', 'Sweden', 'Canada', ..., 'Lithuania', 'Norway', NaN, 'Canada', 'Russia']
Length: 4627
Categories (43, object): ['Austria', 'Belarus', 'Belgium', 'Brazil', ..., 'USSR', 'Ukraine', 'Venezuela', 'Yugoslavia']

In [228]:
# kluczowa wada csv, to text nie zachowuje typów danych
# zaleta to wszystko można zapisać wszystko, i jest obsługiwane przez
# wszystkie narzędzia(excel, sql, python, etc)
# wady parqueta, to nie wszystko obsługuje parquet, ale jest szybszy
# można użyć picle(binarne, z 'c', natywny dla pythona, Java, C++),
master.to_pickle(os.path.join('..', 'data', 'interim', 'master.pkl'))

---

In [229]:
scoring = pd.read_csv(os.path.join('..', 'data', 'raw', 'Scoring.csv'))

In [230]:
memory_mib(scoring)

'10.87 MiB'

In [231]:
scoring.shape

(45967, 31)

In [232]:
scoring.columns


Index(['playerID', 'year', 'stint', 'tmID', 'lgID', 'pos', 'GP', 'G', 'A',
       'Pts', 'PIM', '+/-', 'PPG', 'PPA', 'SHG', 'SHA', 'GWG', 'GTG', 'SOG',
       'PostGP', 'PostG', 'PostA', 'PostPts', 'PostPIM', 'Post+/-', 'PostPPG',
       'PostPPA', 'PostSHG', 'PostSHA', 'PostGWG', 'PostSOG'],
      dtype='object')

In [233]:
scoring.dtypes


playerID     object
year          int64
stint         int64
tmID         object
lgID         object
pos          object
GP          float64
G           float64
A           float64
Pts         float64
PIM         float64
+/-         float64
PPG         float64
PPA         float64
SHG         float64
SHA         float64
GWG         float64
GTG         float64
SOG         float64
PostGP      float64
PostG       float64
PostA       float64
PostPts     float64
PostPIM     float64
Post+/-     float64
PostPPG     float64
PostPPA     float64
PostSHG     float64
PostSHA     float64
PostGWG     float64
PostSOG     float64
dtype: object

In [234]:
scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45967 entries, 0 to 45966
Data columns (total 31 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   playerID  45967 non-null  object 
 1   year      45967 non-null  int64  
 2   stint     45967 non-null  int64  
 3   tmID      45967 non-null  object 
 4   lgID      45967 non-null  object 
 5   pos       45385 non-null  object 
 6   GP        45699 non-null  float64
 7   G         45699 non-null  float64
 8   A         45699 non-null  float64
 9   Pts       45699 non-null  float64
 10  PIM       45699 non-null  float64
 11  +/-       36265 non-null  float64
 12  PPG       37748 non-null  float64
 13  PPA       23040 non-null  float64
 14  SHG       37744 non-null  float64
 15  SHA       23214 non-null  float64
 16  GWG       36567 non-null  float64
 17  GTG       28106 non-null  float64
 18  SOG       36364 non-null  float64
 19  PostGP    19153 non-null  float64
 20  PostG     19153 non-null  fl

In [235]:
scoring.sample(n=10)

,playerID,year,stint,tmID,lgID,pos,GP,G,A,Pts,...,PostA,PostPts,PostPIM,Post+/-,PostPPG,PostPPA,PostSHG,PostSHA,PostGWG,PostSOG
20977,kobasch01,2006,1,CAL,NHL,R,40.0,4.0,13.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24597,lukowmo01,1977,1,HOU,WHA,L,80.0,40.0,35.0,75.0,...,2.0,3.0,17.0,-3.0,NaN,NaN,NaN,NaN,0.0,NaN
8950,davisbi01,1977,1,WIJ,WHA,D,12.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4167,bradyne01,1993,1,DAL,NHL,C,5.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22991,lavoido01,1992,1,OTT,NHL,D,2.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40588,strudja01,1998,1,VAN,NHL,D,65.0,0.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8118,crawfma01,1986,1,VAN,NHL,L,21.0,0.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8444,cunniga01,1973,1,EDO,WHA,D,2.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44759,willija01,2011,1,PIT,NHL,C,8.0,1.0,1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17591,horacto01,1990,1,PHI,NHL,L,34.0,3.0,6.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [236]:
scoring_original = scoring.copy()

In [237]:
scoring = scoring.loc[(scoring['year'] > 1980) & (scoring['lgID'] == 'NHL')]

In [238]:
scoring.shape

(27914, 31)

In [239]:
scoring.iloc[:, [0, 1, 3, 6, 7, 8, 9, 14]].columns

Index(['playerID', 'year', 'tmID', 'GP', 'G', 'A', 'Pts', 'SHG'], dtype='object')

In [240]:
scoring = scoring.filter(regex='^(?!(post|PP|SH)).*')

In [241]:
scoring = scoring.iloc[:, [0, 1, 3, 6, 7, 8, 9, 14]]

In [242]:
scoring.columns

Index(['playerID', 'year', 'tmID', 'GP', 'G', 'A', 'Pts', 'SOG'], dtype='object')

In [243]:
scoring['tmID'].value_counts()

tmID
PIT    1133
NYI    1128
LAK    1125
TOR    1123
NYR    1123
BOS    1118
PHI    1102
CAL    1088
EDM    1086
WAS    1084
CHI    1083
STL    1079
VAN    1065
BUF    1052
MTL    1052
DET    1004
NJD     989
TBL     715
SJS     679
OTT     673
FLO     656
DAL     637
HAR     598
PHO     557
COL     548
WIN     543
QUE     537
MNS     470
CAR     464
NAS     463
CBS     423
ANA     422
ATL     410
MIN     384
AND     224
COR      39
WPG      38
Name: count, dtype: int64

In [244]:
scoring.loc[:, "tmID"] =  pd.Categorical(scoring['tmID'])

In [245]:
scoring.reset_index().head()

,index,playerID,year,tmID,GP,G,A,Pts,SOG
0,0,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,1,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
2,2,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
3,3,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
4,7,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [246]:
scoring.reset_index(drop=True).head()

,playerID,year,tmID,GP,G,A,Pts,SOG
0,aaltoan01,1997,ANA,3.0,0.0,0.0,0.0,1.0
1,aaltoan01,1998,ANA,73.0,3.0,5.0,8.0,61.0
2,aaltoan01,1999,ANA,63.0,7.0,11.0,18.0,102.0
3,aaltoan01,2000,ANA,12.0,1.0,1.0,2.0,18.0
4,abdelju01,2007,DET,2.0,0.0,0.0,0.0,6.0


In [247]:
scoring.reset_index(drop=True, inplace=True)

In [260]:
scoring.to_pickle(os.path.join('..', 'data', 'interim', 'scoring.pkl'))

/tmp/ipykernel_12726/1817049215.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_pickle except for the argument 'path' will be keyword-only.
  scoring.to_pickle(scoring, os.path.join('..', 'data', 'interim', 'scoring.pkl'))


ValueError: Unrecognized compression type: ../data/interim/scoring.pkl
Valid compression types are ['infer', None, 'bz2', 'gzip', 'tar', 'xz', 'zip', 'zstd']

---

In [249]:
teams = pd.read_csv(os.path.join('..', 'data', 'raw', 'Teams.csv'))

In [250]:
teams.shape

(1519, 27)

In [251]:
teams.columns

Index(['year', 'lgID', 'tmID', 'franchID', 'confID', 'divID', 'rank',
       'playoff', 'G', 'W', 'L', 'T', 'OTL', 'Pts', 'SoW', 'SoL', 'GF', 'GA',
       'name', 'PIM', 'BenchMinor', 'PPG', 'PPC', 'SHA', 'PKG', 'PKC', 'SHF'],
      dtype='object')

In [252]:
teams.sample(n=10)

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
637,1976,NHL,VAN,VAN,CC,SM,4,NaN,80,25,...,294,Vancouver Canucks,1078.0,22.0,50.0,263.0,7.0,58.0,258.0,7.0
504,1972,NHL,CHI,CHI,NaN,WD,1,F,78,42,...,225,Chicago Black Hawks,864.0,4.0,52.0,244.0,4.0,51.0,255.0,10.0
442,1967,NHL,MTL,MTL,NaN,ED,1,SC,74,42,...,167,Montreal Canadiens,698.0,NaN,50.0,245.0,3.0,34.0,205.0,7.0
206,1931,NHL,NYR,NYR,NaN,AM,1,F,48,23,...,112,New York Rangers,511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1422,2008,NHL,PIT,PIT,EC,AT,2,SC,82,45,...,239,Pittsburgh Penguins,1114.0,8.0,62.0,360.0,13.0,60.0,347.0,7.0
936,1989,NHL,WIN,PHO,CC,SM,3,DSF,80,37,...,290,Winnipeg Jets,1639.0,10.0,65.0,347.0,7.0,57.0,333.0,14.0
126,1924,NHL,HAM,BKN,NaN,NaN,1,NHLF,30,19,...,60,Hamilton Tigers,335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767,1981,NHL,WAS,WAS,WA,PT,5,NaN,80,26,...,338,Washington Capitals,1932.0,14.0,93.0,418.0,11.0,67.0,309.0,8.0
465,1969,NHL,MNS,DAL,NaN,WD,3,QF,76,19,...,257,Minnesota North Stars,1008.0,20.0,57.0,274.0,7.0,65.0,284.0,10.0
338,1950,NHL,MTL,MTL,NaN,NaN,3,F,70,25,...,184,Montreal Canadiens,835.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519 entries, 0 to 1518
Data columns (total 27 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        1519 non-null   int64  
 1   lgID        1519 non-null   object 
 2   tmID        1519 non-null   object 
 3   franchID    1519 non-null   object 
 4   confID      902 non-null    object 
 5   divID       1171 non-null   object 
 6   rank        1519 non-null   int64  
 7   playoff     936 non-null    object 
 8   G           1519 non-null   int64  
 9   W           1519 non-null   int64  
 10  L           1519 non-null   int64  
 11  T           1309 non-null   float64
 12  OTL         358 non-null    float64
 13  Pts         1519 non-null   int64  
 14  SoW         210 non-null    float64
 15  SoL         210 non-null    float64
 16  GF          1519 non-null   int64  
 17  GA          1519 non-null   int64  
 18  name        1519 non-null   object 
 19  PIM         1409 non-null  

In [254]:
teams = teams.loc[(teams['year'] > 1980) & (teams['lgID'] == 'NHL')]

In [256]:
teams

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
748,1981,NHL,BOS,BOS,WA,AD,2,DF,80,43,...,285,Boston Bruins,1266.0,4.0,65.0,289.0,7.0,54.0,291.0,11.0
749,1981,NHL,BUF,BUF,WA,AD,3,DSF,80,39,...,273,Buffalo Sabres,1425.0,18.0,63.0,301.0,10.0,57.0,281.0,4.0
750,1981,NHL,CAL,CAL,CC,SM,3,DSF,80,29,...,345,Calgary Flames,1331.0,22.0,78.0,290.0,9.0,72.0,328.0,8.0
751,1981,NHL,CHI,CHI,CC,NO,4,CF,80,30,...,363,Chicago Black Hawks,1775.0,12.0,78.0,315.0,12.0,93.0,383.0,13.0
752,1981,NHL,COR,NJD,CC,SM,5,NaN,80,18,...,362,Colorado Rockies,1138.0,20.0,46.0,305.0,15.0,63.0,268.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2011,NHL,TBL,TBL,EC,SE,3,NaN,82,38,...,281,Tampa Bay Lightning,865.0,16.0,41.0,269.0,12.0,59.0,284.0,2.0
1515,2011,NHL,TOR,TOR,EC,NE,4,NaN,82,35,...,264,Toronto Maple Leafs,824.0,16.0,49.0,267.0,6.0,55.0,242.0,5.0
1516,2011,NHL,VAN,VAN,WC,NW,1,CQF,82,51,...,198,Vancouver Canucks,1049.0,10.0,57.0,288.0,4.0,40.0,286.0,7.0
1517,2011,NHL,WAS,WAS,EC,SE,2,CSF,82,42,...,230,Washington Capitals,767.0,16.0,41.0,245.0,10.0,49.0,266.0,3.0


In [257]:
teams.nunique()

year           30
lgID            1
tmID           37
franchID       30
confID          4
divID          10
rank            7
playoff         7
G               4
W              49
L              50
T              19
OTL            18
Pts            90
SoW            15
SoL            12
GF            209
GA            214
name           37
PIM           594
BenchMinor     22
PPG            85
PPC           244
SHA            22
PKG            85
PKC           249
SHF            28
dtype: int64

In [258]:
teams.loc[:, 'tmID'] = pd.Categorical(teams['tmID'])

In [259]:
teams.to_pickle(os.path.join('..', 'data', 'interim', 'teams.pkl'))

---

In [262]:
team_splits = pd.read_csv(os.path.join('..', 'data', 'raw', 'TeamSplits.csv'))

In [263]:
team_splits.shape

(1519, 43)

In [265]:
team_splits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519 entries, 0 to 1518
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1519 non-null   int64  
 1   lgID    1519 non-null   object 
 2   tmID    1519 non-null   object 
 3   hW      1519 non-null   int64  
 4   hL      1519 non-null   int64  
 5   hT      1309 non-null   float64
 6   hOTL    358 non-null    float64
 7   rW      1519 non-null   int64  
 8   rL      1519 non-null   int64  
 9   rT      1309 non-null   float64
 10  rOTL    358 non-null    float64
 11  SepW    2 non-null      float64
 12  SepL    2 non-null      float64
 13  SepT    0 non-null      float64
 14  SepOL   2 non-null      float64
 15  OctW    1199 non-null   float64
 16  OctL    1199 non-null   float64
 17  OctT    989 non-null    float64
 18  OctOL   358 non-null    float64
 19  NovW    1358 non-null   float64
 20  NovL    1358 non-null   float64
 21  NovT    1148 non-null   float64
 22  

In [266]:
team_splits.head()

,year,lgID,tmID,hW,hL,hT,hOTL,rW,rL,rT,...,FebT,FebOL,MarW,MarL,MarT,MarOL,AprW,AprL,AprT,AprOL
0,1909,NHA,COB,2,4,0.0,NaN,2,4,0.0,...,0.0,NaN,1.0,4.0,0.0,NaN,NaN,NaN,NaN,NaN
1,1909,NHA,HAI,3,3,0.0,NaN,1,5,0.0,...,0.0,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,1909,NHA,LES,2,4,0.0,NaN,0,6,0.0,...,0.0,NaN,1.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN
3,1909,NHA,MOS,3,2,1.0,NaN,0,6,0.0,...,0.0,NaN,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
4,1909,NHA,MOW,6,0,0.0,NaN,5,1,0.0,...,0.0,NaN,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [264]:
team_splits.columns

Index(['year', 'lgID', 'tmID', 'hW', 'hL', 'hT', 'hOTL', 'rW', 'rL', 'rT',
       'rOTL', 'SepW', 'SepL', 'SepT', 'SepOL', 'OctW', 'OctL', 'OctT',
       'OctOL', 'NovW', 'NovL', 'NovT', 'NovOL', 'DecW', 'DecL', 'DecT',
       'DecOL', 'JanW', 'JanL', 'JanT', 'JanOL', 'FebW', 'FebL', 'FebT',
       'FebOL', 'MarW', 'MarL', 'MarT', 'MarOL', 'AprW', 'AprL', 'AprT',
       'AprOL'],
      dtype='object')

In [269]:
team_splits = team_splits.loc[(team_splits["lgID"] == 'NHL') & (team_splits["year"] >= 1980)]

In [270]:
team_splits

,year,lgID,tmID,hW,hL,hT,hOTL,rW,rL,rT,...,FebT,FebOL,MarW,MarL,MarT,MarOL,AprW,AprL,AprT,AprOL
727,1980,NHL,BOS,26,10,4.0,NaN,11,20,9.0,...,2.0,NaN,8.0,4.0,3.0,NaN,1.0,2.0,0.0,NaN
728,1980,NHL,BUF,21,7,12.0,NaN,18,13,9.0,...,3.0,NaN,8.0,6.0,1.0,NaN,1.0,1.0,1.0,NaN
729,1980,NHL,CAL,25,5,10.0,NaN,14,22,4.0,...,1.0,NaN,6.0,6.0,2.0,NaN,2.0,0.0,0.0,NaN
730,1980,NHL,CHI,21,11,8.0,NaN,10,22,8.0,...,4.0,NaN,4.0,5.0,4.0,NaN,1.0,1.0,1.0,NaN
731,1980,NHL,COR,15,16,9.0,NaN,7,29,4.0,...,2.0,NaN,4.0,9.0,1.0,NaN,0.0,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,2011,NHL,TBL,25,14,NaN,2.0,13,22,NaN,...,NaN,2.0,7.0,7.0,NaN,1.0,2.0,1.0,NaN,1.0
1515,2011,NHL,TOR,18,16,NaN,7.0,17,21,NaN,...,NaN,1.0,5.0,8.0,NaN,2.0,1.0,1.0,NaN,1.0
1516,2011,NHL,VAN,27,10,NaN,4.0,24,12,NaN,...,NaN,4.0,9.0,5.0,NaN,1.0,2.0,1.0,NaN,0.0
1517,2011,NHL,WAS,26,11,NaN,4.0,16,21,NaN,...,NaN,1.0,8.0,5.0,NaN,3.0,2.0,1.0,NaN,0.0


In [271]:
cols_to_drop = team_splits.columns[3:11]

In [272]:
team_splits.drop(columns=cols_to_drop, inplace=True)

In [274]:
team_splits.columns

Index(['year', 'lgID', 'tmID', 'SepW', 'SepL', 'SepT', 'SepOL', 'OctW', 'OctL',
       'OctT', 'OctOL', 'NovW', 'NovL', 'NovT', 'NovOL', 'DecW', 'DecL',
       'DecT', 'DecOL', 'JanW', 'JanL', 'JanT', 'JanOL', 'FebW', 'FebL',
       'FebT', 'FebOL', 'MarW', 'MarL', 'MarT', 'MarOL', 'AprW', 'AprL',
       'AprT', 'AprOL'],
      dtype='object')

In [275]:
team_splits.drop(columns="lgID", inplace=True)

In [276]:
team_splits.shape

(792, 34)

In [277]:
team_splits.loc[:, 'tmID'] = pd.Categorical(team_splits['tmID'])

In [279]:
team_splits.to_pickle(os.path.join('..', 'data', 'interim', 'team_splits.pkl'))
